In [0]:
answers_file = "bio_questions.csv"
df_ques = pd.read_csv(answers_file)
saved_column = df_ques.Body #you can also use df['column_name']
saved_column.fillna(" ")
tfidf_bio_ques = get_tfidf(saved_column)

saved_column.head()

bio_ques_rec_dict = defaultdict(list)
for i in tqdm.tqdm(range(saved_column.shape[0])):
    for index, score in find_similar(tfidf_bio_ques, i):
        if df_ques['Id'][i] in bio_ques_rec_dict:
            (bio_ques_rec_dict[df_ques['Id'][i]]).append((df_ques['Id'][index],score))
        else:
            bio_ques_rec_dict[df_ques['Id'][i]] = [(df_ques['Id'][index],score)]
save_obj(bio_ques_rec_dict,"tf_idf_bio_ques_rec")

In [0]:

import pandas as pd
postlinksDF = pd.read_csv('postlinks.csv')

In [4]:
import tqdm 
#final list of dict to keep track of related post ids and answer ids for each question id
postLinks = []
#to keep track of questions that were already encountered
posts = [] 

for index, row in tqdm.tqdm(postlinksDF.iterrows()):
    temp = {'postID': None,'relatedID': [],'answerID': [] } 
    temp['postID'] = row['PostId']
    if(row['PostId'] in posts and len(posts)!=0):
        for i in range(0,len(posts)):
            if(row['PostId'] == postLinks[i]['postID']):
                postLinks[i]['relatedID'].append(row["RelatedPostId"])
                postLinks[i]['answerID'].append(row["Id"])
    else:  
        temp['relatedID'].append(row["RelatedPostId"])
        temp['answerID'].append(row["Id"])   
        posts.append(temp['postID'])
        postLinks.append(temp)


0it [00:00, ?it/s]
207it [00:00, 2046.87it/s]
323it [00:00, 1653.96it/s]
403it [00:00, 1252.33it/s]
482it [00:00, 1012.87it/s]
558it [00:00, 847.34it/s] 
630it [00:00, 743.91it/s]
698it [00:00, 651.25it/s]
761it [00:00, 618.53it/s]
822it [00:01, 535.69it/s]
878it [00:01, 532.80it/s]
932it [00:01, 417.38it/s]
979it [00:01, 401.13it/s]
1023it [00:01, 372.32it/s]
1063it [00:01, 325.85it/s]
1099it [00:01, 268.78it/s]
1130it [00:02, 266.44it/s]
1161it [00:02, 269.68it/s]
1190it [00:02, 239.11it/s]
1216it [00:02, 210.07it/s]
1239it [00:02, 204.43it/s]
1261it [00:02, 185.98it/s]
1281it [00:02, 164.77it/s]
1304it [00:03, 176.01it/s]
1327it [00:03, 182.31it/s]
1360it [00:03, 210.41it/s]
1402it [00:03, 243.16it/s]
1430it [00:03, 224.47it/s]
1456it [00:03, 222.63it/s]
1481it [00:03, 174.26it/s]
1502it [00:04, 145.78it/s]
1520it [00:04, 146.76it/s]
1540it [00:04, 156.38it/s]
1558it [00:04, 147.52it/s]
1583it [00:04, 163.73it/s]
1611it [00:04, 179.73it/s]
1632it [00:04, 184.46it/s]
1652it [00:04, 

In [0]:
import pickle as pickle

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

tf_idf_bio_ques_rec = load_obj("tf_idf_bio_ques_rec")
#tf_idf_bio_answers_rec = load_obj("tf_idf_bio_answers_rec")

In [0]:
def evaluate(recommended):
    score  = 0
    for i in range(0,len(postLinks)):
        present = 0 
        top = 0
        count = len(postLinks[i]['relatedID'])
        if postLinks[i]['postID'] in recommended.keys():
            idx = 0
            for rec in recommended[postLinks[i]['postID']]:
                idx+=1
                if rec[0] in postLinks[i]['relatedID']:
                    present+=1
                    if (idx >= int(len(postLinks[i]['relatedID'])/2)):
                        top+=1
            if(present):
                score+= (0.1* (present/count)+ 0.9*(top/present))
    return (score/len(postLinks))

In [0]:
evaluate(tf_idf_bio_answers_rec)*100

35.96668285457064

In [8]:
evaluate(tf_idf_bio_ques_rec)*100

27.31680282246091